# Simulate rhythmic growth with a thermocycler

2023/09/12    Initial write-up

2025/07/31    Imposed injection cycle

The thermocycler module has the convenient functionality of opening or closing the lid on command (by a chunk of code). With this we can only leave the tubes open to air when actively pipetting (on the scale of minutes) and closed for the rest of the time. This will significantly reduce evaporation compared to leaving the tubes open at all times without the help of any oil overlay.

Previous experimental results (FP) suggest that proteins are damaged after 3 days of simulated growth with oil, but U-KaiC prepped worked fine. So likely oil is the problem here. With the thermocyler we can minimize evaporation without the help of oil which will likely improve the quality of the reaction. I will keep the samples in the thermocyler too so no need to add water back -- this will streamline the protocol and reduce the amount of plastic waste.

In [1]:
import numpy as np
import opentrons.execute
from opentrons import protocol_api
from src.pipette_viscous import transfer_viscous,aspirate_viscous, dispense_viscous, calibrated_viscous
import src.scheduler as scheduler
from src.evap_standard_curve import *
import random, time

# reset seed everytime
random.seed(time.time())

In [2]:
# >= v2.13 is required for the thermocycler module
protocol = opentrons.execute.get_protocol_api("2.22")

protocol.set_rail_lights(False)
protocol.home()

## A description of the protocol

The volume of the system grows exponentially with the exponent determined by a single parameter doubling time. Protein addition rate matches exactly the volume expansion rate so that the total concentration of each protein stays at a steady state. Only unphosphorylated KaiC can be added. In time interval $\Delta t$ the volume becomes

$$
V(\Delta t) = V_0 2^{\Delta t/g}
$$

which means the increase in volume is 

$$
\Delta V = V(\Delta t) - V_0 = V_0(2^{\Delta t/g} - 1) \approx V_0\cdot\ln(2)\Delta t/g
$$

Given time interval between growth steps and the increase in volume in the duration one finds the initial volume to be

$$
V_0 = \frac{\Delta V\cdot g}{\ln(2)\Delta t}
$$

In [3]:
g = 6*60    # minutes. Simulated doubing (generation) time
# g = 40

dt_sample = 4*60    # minutes. sampling time interval. Samples are taken before the perturbation i.e. dilution
# dt_sample = 20    # test. Total run time would be 8 hours + relaxation time

dt_perturb = 2*60    # minutes. perturbation time interval
# dt_perturb = 10

t_relax = 10    # minutes. wait for this amount of time before first perturbation. First peak is always higher
# t_relax = 10

v_perturb = 6    # ul. Total increase in volume at each perturbation.
# Contribution from each KaiA stock will be 2 ul

v_sample = 2    # ul

v_discard = 2*v_perturb - v_sample    # 10 ul

# calculate initial volume
v0 = v_perturb*g/(np.log(2)*dt_perturb)
v0 = np.round(v0)

print("Initial volume =", v0 + v_sample, "ul")

Initial volume = 28.0 ul


## Define labware

In [4]:
rack = protocol.load_labware("eppendorf_24_aluminumblock_1500ul", '6')

# use opentrons 20 ul tips which have the same dimensions as the GEB
tip_rack_1 = protocol.load_labware("geb_taller_96_tiprack_10ul", '4')
tip_rack_2 = protocol.load_labware("geb_taller_96_tiprack_10ul", '5')

tc_mod = protocol.load_module(module_name='thermocyclerModuleV2')
plate = tc_mod.load_labware(name='biorad_96_wellplate_200ul_calibrated')

pipette = protocol.load_instrument("p10_single", "left", tip_racks = [tip_rack_1, tip_rack_2])

In [5]:
RXN_TUBE = plate.wells_by_name()["C3"]
A_TUBE = plate.wells_by_name()["E3"]
B2_TUBE = plate.wells_by_name()["E4"]    # twice the conc. but smaller volume needed
U2_TUBE = plate.wells_by_name()["E5"]
R_TUBE = plate.wells_by_name()["F3"]

DIS_TUBE = rack.wells_by_name()["A1"]

To make the reaction mixture, mix 1.5 uM of KaiA, 3.5 uM of KaiB, and 3.5 uM of KaiC in Kai Storage Buffer (refer to Rust Lab Dropbox for recipe. It is a Tris-HCl-based buffer at pH 8 with Mg++) with 1 mM of ATP. The reaction mixture will be in well C3 of the plate in the thermocycler

I will also prepare 3x Kai protein solutions in three separate wells, E3, E4, and E5. KaiC will be unphosphorylated before adding to the tube. 30 hours before first injection (that is 6 hours before the start of the program), I will mix KaiC with 1 mM of ATP (10.5 uM KaiC will consume around 0.2 mM after 30 hours and 0.67 mM after the full time coure of injection. That's probably fine. But it's also possible that protein starts to crash out with 0.3 mM of ATP. I will prepare R buffer with 2 mM of ATP).

Load well A7 to A10 (25 wells in total) with 3 ul of SDS samping buffer and 7ul of water (12 ul in total together with 2 ul of sample. The volume will be smaller by the end of the run due to evaporation)

Wipe down the rubber seal with ethanol before the run!

Volumes:

V(A stock tubes) = 50 ul (safe volume) + 2*36 ul = 122 ul

V(2xB or 2xC stock tubes) = 50 ul (safe volume) + 2*18 ul = 86 ul

V(R buffer) = 50 ul + 4*18 ul = 122 ul

This is close to the maximum volume (150 ul)! Is this a concern? Would it be helpful to turn off the lid heater (at 37 C) to minimize heating the protein?

![plate layout](fig/plate_layout_for_repeated_perturbations3.jpg)

<div class="alert alert-warning">
    <h2>STOP!</h2>
    <br />
    Have you placed the required labware?<br />
    Have you put the right volume of liquid in required position?<br />
    Remember to add the dye before the robot run!!
</div>

In [6]:
# constants

SAMPLE_OFFSET = 48    # samples starting from the second half of the plate
# RM_OFFSET = -0.5    # estimate of the residual volume on the tip. If you want to remove solution v, put v - RM_OFFSET in the code

## Operations

In [7]:
def sample_and_discard(idx, v_s, v_d, followed_by):
    "take a sample from the tube to a 96-well plate"
    
    tc_mod.open_lid()

    # current well
    cur_well = plate.wells()[idx + SAMPLE_OFFSET]

    # pipette the sample
    transfer_viscous(pipette, protocol, v_s, RXN_TUBE, cur_well, 
                     asp_height=1.5, 
                     disp_height=0.5,
                     delay=0, 
                     if_touch_tip=False, 
                     if_mix=True, 
                     if_blowout=True)
    
    # when there's no perturbation e.g. at time 0 and on the final day
    # don't discard additional materials
    if v_d > 1e-4:
        aspirate_viscous(pipette, protocol, v_d, RXN_TUBE, 
                         asp_height=1.5, 
                         delay=0, 
                         if_touch_tip=False)
        pipette.dispense(v_d, DIS_TUBE)
        pipette.drop_tip()
      
    if not followed_by:
        tc_mod.close_lid()

# for logging
str_sample_and_discard = "Sample and mix with loading buffer, and discard from the reaction"
str0 = "Sample and mix with loading buffer"

        
def grow_in_window(idx, v_g, preceded_by):
    "dilute the reaction with U-KaiC, KaiA, and KaiB by volume v_g"
    
    if not preceded_by:
        tc_mod.open_lid()
    
    v_g_per_tube = np.round(v_g/3, 2)
    
    # shuffle the order each time pipetting happens
    # this should help to keep the end volume the same
    stock_tubes = [A_TUBE, B2_TUBE, U2_TUBE]
    random.shuffle(stock_tubes)
    
    for from_tube in stock_tubes:
        # it's also okay to mix KaiA and KaiB in one tube, then all conc are 2x
        # do not dispense at the bottom. Defaults are 1mm from the bottom
        aspirate_viscous(pipette, protocol, calibrated_viscous(v_g_per_tube), from_tube, 
                         asp_height=1.5, 
                         delay=0, 
                         if_touch_tip=False)
        dispense_viscous(pipette, protocol, calibrated_viscous(v_g_per_tube), RXN_TUBE, 
                         disp_height=3, 
                         delay=0, 
                         if_mix=True, 
                         mix_rate=0.2, 
                         if_blowout=False)
        
    tc_mod.close_lid()
    
str_grow_in = "Inject KaiA, 2xKaiB, and 2xU-KaiC stocks"

def grow_out_of_window(idx, v_g, preceded_by):
    "dilute the reaction with KaiA and buffer by volume v_g"
    
    if not preceded_by:
        tc_mod.open_lid()
    
    v_g_per_tube = np.round(v_g/3, 2)
    
    # shuffle the order each time pipetting happens
    # this should help to keep the end volume the same
    stock_tubes = [A_TUBE, R_TUBE, R_TUBE]
    random.shuffle(stock_tubes)
    
    for from_tube in stock_tubes:
        # it's also okay to mix KaiA and KaiB in one tube, then all conc are 2x
        # do not dispense at the bottom. Defaults are 1mm from the bottom
        aspirate_viscous(pipette, protocol, calibrated_viscous(v_g_per_tube), from_tube, 
                         asp_height=1.5, 
                         delay=0, 
                         if_touch_tip=False)
        dispense_viscous(pipette, protocol, calibrated_viscous(v_g_per_tube), RXN_TUBE, 
                         disp_height=3, 
                         delay=0, 
                         if_mix=True, 
                         mix_rate=0.2, 
                         if_blowout=False)
        
    tc_mod.close_lid()
    
str_grow_out_of = "Inject KaiA stock and buffer"

## Instructions

In [8]:
tc_mod.close_lid()
tc_mod.set_lid_temperature(temperature=37)    # lid temp set point >= 37
tc_mod.set_block_temperature(temperature=30)

In [9]:
# clear whatever schedule there might have been
scheduler.drop()

# sampling
N1 = 25    # number of operations
# in day 2 to 4 sampling steps will be followed by adding steps
fb_vec = [ i < 18 for i in range(N1) ]
vd_vec = [ v_discard if i > 0 and i <= 18 else 0 for i in range(N1) ]

# first 24 hours is incubation
scheduler.cat(time_vec = np.arange(N1)*dt_sample + t_relax,    # minutes. sample for 4 days including time 0
             func_vec = [sample_and_discard]*N1,
             param_vec = [ (i, v_sample, vd_vec[i], fb_vec[i]) for i in range(N1) ],
             str_vec = [str0] + [str_sample_and_discard]*(N1 - 1),
             n_tip_vec = [ 2 if i > 0 and i <= 18 else 1 for i in range(N1) ])

# perturbations
N2 = 36    # within a 3-day window. no perturbation after the last sample of the 4th day is taken

f_vec = [grow_in_window]*6 + [grow_out_of_window]*6
s_vec = [str_grow_in]*6 + [str_grow_out_of]*6
f_perturb_vec = f_vec + f_vec + f_vec
s_perturb_vec = s_vec + s_vec + s_vec

# even number steps will be preceded by sampling steps (the 1st, 3rd, etc.)
pb_vec = [ i%2 == 0 for i in range(N2) ]
scheduler.cat(time_vec = np.arange(N2)*dt_perturb + t_relax,
             func_vec = f_perturb_vec,
             param_vec = [ (i, v_perturb, pb_vec[i]) for i in range(N2) ],
             str_vec = s_perturb_vec,
             n_tip_vec=[3]*N2)

scheduler.report(unit="hours")

The protocol requires a total of 151 tip(s)

At 0.17 hours, Sample and mix with loading buffer, with params (0, 2, 0, True)
At 4.17 hours, Sample and mix with loading buffer, and discard from the reaction, with params (1, 2, 10, True)
At 8.17 hours, Sample and mix with loading buffer, and discard from the reaction, with params (2, 2, 10, True)
At 12.17 hours, Sample and mix with loading buffer, and discard from the reaction, with params (3, 2, 10, True)
At 16.17 hours, Sample and mix with loading buffer, and discard from the reaction, with params (4, 2, 10, True)
At 20.17 hours, Sample and mix with loading buffer, and discard from the reaction, with params (5, 2, 10, True)
At 24.17 hours, Sample and mix with loading buffer, and discard from the reaction, with params (6, 2, 10, True)
At 28.17 hours, Sample and mix with loading buffer, and discard from the reaction, with params (7, 2, 10, True)
At 32.17 hours, Sample and mix with loading buffer, and discard from the reaction, with params

In [ ]:
log_fn = "log/20250803_rhythmic_growth_g6_test_full_timecourse.log"
scheduler.run(protocol, log_fn)

In [11]:
tc_mod.deactivate_block()
tc_mod.deactivate_lid()
tc_mod.open_lid()
protocol.set_rail_lights(False)